In [18]:
import pandas as pd
import numpy as np
import json, glob
import itertools
import os
from pathlib import Path

In [8]:
exp_dir = Path('../data/experiments/11-27-2021/13_36')
manifest = exp_dir / 'test.json'
res_file = exp_dir / 'all_results.json'

In [9]:
results = json.load(open(res_file))
prompts = json.load(open(manifest))
assert len(results) == len(prompts)
len(prompts)

6

In [20]:
a = list(itertools.product(['-2=>', '-1=>', 'False=>', 'True=>'], ['-2', '-1', 'False', 'True']))
a = [''.join(b) for b in a]

In [32]:
def split_prob(prob, keep_filename=True):
    tmp = prob.split('Prompt-Summarization/data/')[1]
    if keep_filename:
        return tmp
    return os.path.split(tmp)[0]

In [34]:
probs = {split_prob(prompt, False) for prompt in prompts}
probs

{'competition/2000',
 'studio21_generated/competition/2002',
 'studio21_generated/test/competition/3000'}

In [35]:
questions = [split_prob(prompt) for prompt in prompts]
questions

['studio21_generated/competition/2002/expert.txt',
 'studio21_generated/competition/2002/question.txt',
 'studio21_generated/test/competition/3000/expert.txt',
 'studio21_generated/test/competition/3000/question.txt',
 'competition/2000/expert.txt',
 'competition/2000/question.txt']

In [79]:
acc = pd.DataFrame(0, index=questions, columns=[-2, -1, False, True])
for res, prompt in zip(results.values(), questions):
    i = 0
    for r in res[0]:
        i += 1
        acc.loc[prompt, r] +=1

In [80]:
acc

,-2,-1,False,True
studio21_generated/competition/2002/expert.txt,0,19,0,0
studio21_generated/competition/2002/question.txt,1,0,0,0
studio21_generated/test/competition/3000/expert.txt,0,5,0,1
studio21_generated/test/competition/3000/question.txt,0,2,4,0
competition/2000/expert.txt,0,0,3,16
competition/2000/question.txt,0,19,0,0


In [81]:
acc_df = acc.copy(deep=False)
acc_df['acc'] = 100*acc_df[True]/acc_df.sum(axis=1)
acc_df

,-2,-1,False,True,acc
studio21_generated/competition/2002/expert.txt,0,19,0,0,0.000000
studio21_generated/competition/2002/question.txt,1,0,0,0,0.000000
studio21_generated/test/competition/3000/expert.txt,0,5,0,1,16.666667
studio21_generated/test/competition/3000/question.txt,0,2,4,0,0.000000
competition/2000/expert.txt,0,0,3,16,84.210526
competition/2000/question.txt,0,19,0,0,0.000000


In [82]:
strict_accuracy = 100*sum(acc_df['acc'] >= 100)/len(acc_df)
strict_accuracy

0.0

In [78]:
def detect_type(prompt):
    types = ['introductory', 'interview', 'competition']
    for t in types:
        if t in prompt:
            return t

In [84]:
diff_df = acc.groupby(detect_type).sum()
diff_df

,-2,-1,False,True
competition,1,45,7,17


In [85]:
diff_df['acc'] = 100*diff_df[True]/diff_df.sum(axis=1)
diff_df

,-2,-1,False,True,acc
competition,1,45,7,17,24.285714


In [86]:
def detect_test(prompt):
    t = 'test'
    if t in prompt:
        return t
    return 'train'

In [88]:
test_df = acc.groupby(detect_test).sum()
test_df

,-2,-1,False,True
test,0,7,4,1
train,1,38,3,16


In [89]:
def detect_summary(prompt):
    if prompt.endswith('question.txt'):
        return 'question'

In [98]:
for _, g in acc.groupby(np.arange(len(acc)) // 2):
    print(g)
    print(type(g))

                                                  -2  -1  False  True
studio21_generated/competition/2002/expert.txt     0  19      0     0
studio21_generated/competition/2002/question.txt   1   0      0     0
<class 'pandas.core.frame.DataFrame'>
                                                    -2  -1  False  True
studio21_generated/test/competition/3000/expert...   0   5      0     1
studio21_generated/test/competition/3000/questi...   0   2      4     0
<class 'pandas.core.frame.DataFrame'>
                               -2  -1  False  True
competition/2000/expert.txt     0   0      3    16
competition/2000/question.txt   0  19      0     0
<class 'pandas.core.frame.DataFrame'>


In [101]:
t = acc.iterrows()
for (i, row1), (j, row2) in zip(t, t):
    #print(f'I am row1\n{row1}')
    #print(f'I am row2\n{row2}')
    b = row2 - row1
    print(b)

-2        1
-1      -19
False     0
True      0
dtype: int64
-2       0
-1      -3
False    4
True    -1
dtype: int64
-2        0
-1       19
False    -3
True    -16
dtype: int64


In [106]:
for k, v in itertools.filterfalse(lambda x: int(x[0]) % 2, results.items()):
    k = int(k)
    if questions[k].endswith('expert.txt'):
        summary = v
        original = results[str(k+1)]
    else:
        summary = results[str(k+1)]
        original = v